## Imports

In [1]:
import numpy as np 

## Reading Databases

In [2]:
class Vertex():
    def __init__(self, name):
        self.name = name
        self.edges = []
        self.weights = []
        self.pheromone = [] 
    
    def add(self, other, weight):
        self.edges.append(other)
        self.weights.append(weight)
        self.pheromone.append(10)  # Inicialize pheromone equals 10 to all edges

In [3]:
class Graph():
    def __init__(self):
        self.vertices = dict()
    
    def build_graph(self, file):
        with open(file) as f:
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            for line in lines:
                line = line.split('\t')
                line = list(map(float, line))
                if line[0] not in self.vertices.keys():
                    self.vertices[line[0]] = Vertex(line[0])
                self.vertices[line[0]].add(line[1], line[2])
    
    def update_pheromone(self, origin, destine, cost):
            index = self.vertices[origin].edges.index(destine)
            self.vertices[origin].pheromone[index] += cost
    
    def evaporate_pheromone(self, evaporation_rate):
        for vertex in self.vertices.keys():
            #self.vertices[vertex].pheromone -= np.array(evaporation_rate)
            self.vertices[vertex].pheromone *= np.array(1-evaporation_rate)
            self.vertices[vertex].pheromone = np.clip(self.vertices[vertex].pheromone, a_min=0, a_max=None)
    
    def show_pheromone(self):
        for vertex in self.vertices.keys():
            print(self.vertices[vertex].pheromone)
    
    def reset_pheromone(self):
        for vertex in self.vertices.keys():
            self.vertices[vertex].pheromone = [10 for _ in self.vertices[vertex].pheromone]

In [4]:
file = 'graph1.txt'
G1 = Graph()
G1.build_graph(file)

## ACO 

In [5]:
class Ant():
    def __init__(self, destine):
        self.path = [1]
        self.destine = destine
    
    def show(self):
        for e in self.path[:-1]:
            print(e, end=', ')
        print(self.path[-1])
    
    def step(self, graph, alfa=1, beta=2):
        vertex = self.path[-1]
        edges = graph.vertices[vertex].edges
        pheromone = graph.vertices[vertex].pheromone
        weights = graph.vertices[vertex].weights
        index = [i for (i,e) in enumerate(edges) if e not in self.path]
        if index == []:
            
            return None, 0
        #edges = [e for (i,e) in enumerate(edges) if i in index]
        pheromone = [p for (i,p) in enumerate(pheromone) if i in index]
        v_weights = [w for (i,w) in enumerate(weights) if i in index]
        #print(edges)
        prob_num = np.array(pheromone)**alfa * np.array(v_weights)**beta
        prob = prob_num/np.sum(prob_num)
        chosen = np.random.choice(index, p=prob)
        destination = edges[chosen]
        cost = weights[chosen]
        self.path.append(destination)
        return destination, cost
        
    def walk(self, graph, alfa=1, beta=2):
        total_cost = 0
        destination = 1
        while(destination != self.destine and destination != None):
            destination, cost = self.step(graph, alfa, beta)
            # print(destination, len(self.path))
            ##Test
            ##if len(self.path) <= 99:
            ##     if destination == 100:
            ##            self.path.pop()
            ##            destination = 0
            ##            continue
            ## 
            total_cost += cost

        #self.show()
        if destination == None:
            print("DEAD END ")
            self.path=[1]  # Clear ant path before return 
            return 0, None
        #print("Total lenght = ", len(self.path))
        #print("Total weight = ", total_cost)
        
        # Reset ant path
        path = self.path 
        self.path = [1]  # Clear ant path before return 
        
        return total_cost, path


In [6]:
class ACO():
    def __init__(self, graph=None, number_of_ants=20, max_iterations=100, evaporation_rate=0.05, alfa=1, beta=2):
        self.number_of_ants = number_of_ants
        self.graph = graph
        self.ants = [Ant(len(self.graph.vertices)) for _ in range(self.number_of_ants)]
        self.max_iterations = max_iterations
        self.evaporation_rate = evaporation_rate
        self.current_iteration = 0
        self.best = (0, None)
        self.alfa = alfa
        self.beta = beta
        
    def iteration(self):
        self.current_iteration += 1
        
        #self.graph.show_pheromone()
        #Evaporate pheromone
        self.graph.evaporate_pheromone(self.evaporation_rate)
        
        for ant in self.ants:
            cost, path = ant.walk(self.graph, self.alfa, self.beta)
            if path == None:
                continue
            
            #Update best 
            if cost > self.best[0]:
                self.best = (cost, path)
        
            #Update pheromone
            for i, vertex in enumerate(path[:-1]):
                origin = vertex
                destine = path[i+1]
                self.graph.update_pheromone(origin, destine, cost/100)
        
        #self.graph.show_pheromone()
        
        print("Best so far: ", self.best[0])
    
    def run(self):
        for i in range(self.max_iterations):
            self.iteration()
    
    def reset(self):
        self.current_iteration = 0
        self.best = (0, None)
        self.graph.reset_pheromone()

## Problem 1

In [28]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.05
sol1 = ACO(G1, number_of_ants, max_iterations, evaporation_rate)

In [31]:
sol1.run()

Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0
Best so far:  970.0


## Problem 2

In [9]:
file = 'graph2.txt'
G2 = Graph()
G2.build_graph(file)

In [23]:
number_of_ants = 20
max_iterations = 150
evaporation_rate = 0.01
alfa = 1
beta = 2
sol2 = ACO(G2, number_of_ants, max_iterations, evaporation_rate, alfa, beta)

In [27]:
sol2.reset()
sol2.run()

Best so far:  116.0
DEAD END 
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
DEAD END 
Best so far:  142.0
DEAD END 
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
DEAD END 
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
Best so far:  142.0
DEAD END 
Best so far:  142.0
Best so far:  142.0
Best so far:  147.0
DEAD END 
DEAD END 
Best so far:  151.0
Best so far:  151.0
Best so far:  151.0
Best so far:  151.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
DEAD END 
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
DEAD END 
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so far:  152.0
Best so fa